In [1]:
import numpy as np
import pandas as pd

In [5]:
df1 = pd.read_csv("C:\\Users\\user\\Desktop\\all-cancelled-flights.csv")
df2 = pd.read_csv("C:\\Users\\user\\Desktop\\cancellation-codes.csv")

In [7]:
merged_df = pd.merge(df1, df2, left_on='CANCELLATION_CODE', right_on='Code', how='left')

In [21]:
# 將合併後的結果寫入新的 CSV 文件
merged_df.to_csv('Merged_Files.csv', index=False)

In [19]:
merged_df.head()

,FLIGHT_DATE,DAY,OP_CARRIER,CARRIER_NAME,OP_CARRIER_FL_NUM,ORIGIN,DISPLAY_AIRPORT_NAME_ORIGIN,DEST,DISPLAY_AIRPORT_NAME_DEST,CANCELLATION_CODE,NUMBER_OF_FLIGHTS,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,LATITUDE_DEST,LONGITUDE_DEST,DISTANCE_LEGEND,Code,Description,Unnamed: 2
0,2020/10/1,Thursday,BA,AnyCompany Blue Airlines,5029,ATL,Atlanta Municipal,MLB,Melbourne Regional,A,1,33.640833,-84.427222,28.102500,-80.643056,250-499 Miles,A,Carrier,NaN
1,2020/10/1,Thursday,BA,AnyCompany Blue Airlines,5069,MLB,Melbourne Regional,ATL,Atlanta Municipal,A,1,28.102500,-80.643056,33.640833,-84.427222,250-499 Miles,A,Carrier,NaN
2,2020/10/1,Thursday,GA,AnyCompany Green Airlines,134,DFW,Dallas Fort Worth Regional,OGG,Kahului Airport,D,1,32.894444,-97.029722,20.900556,-156.433611,2500 Miles and Greater,D,Security,NaN
3,2020/10/1,Thursday,GA,AnyCompany Green Airlines,1960,OGG,Kahului Airport,PHX,Phoenix Sky Harbor International,D,1,20.900556,-156.433611,33.436111,-112.009444,2500 Miles and Greater,D,Security,NaN
4,2020/10/1,Thursday,GA,AnyCompany Green Airlines,2242,LAX,Los Angeles International,KOA,Keahole,D,1,33.942500,-118.408056,19.738889,-156.045556,2500 Miles and Greater,D,Security,NaN


In [27]:
from collections import Counter

df = pd.DataFrame(merged_df)
# 假設你的欄位名稱是 'column1'
# 首先將字串的所有字符統計
df['counts'] = df['CANCELLATION_CODE'].apply(lambda x: Counter(x))

# 計算總長度並計算各字元所佔比例
df['總長度'] = df['CANCELLATION_CODE'].apply(len)

# 創建一個新的DataFrame來表示所有字元的總數
counts_sum = df['counts'].sum()

# 計算比例
proportions = {key: value / df['總長度'].sum() for key, value in counts_sum.items()}
proportions

{'A': 0.37304272810711375,
 'D': 0.012735315783046628,
 'B': 0.5271388673329105,
 'C': 0.08708308877692911}

In [31]:
# 計算每個航班號取消的次數
flight_cancellations = cancelled_flights['OP_CARRIER_FL_NUM'].value_counts()

In [33]:
# 找出取消次數最多的航班號及其取消次數
max_cancelled_flight = flight_cancellations.idxmax()  # 取消次數最多的航班號
max_cancellations = flight_cancellations.max()  # 該航班號的取消次數

In [35]:
# 計算總取消次數
total_cancellations = flight_cancellations.sum()

# 計算該航班號的取消百分比
cancel_percentage = (max_cancellations / total_cancellations) * 100

In [39]:
print(f"取消最多的航班號是：{max_cancelled_flight}")
print(f"取消次數：{max_cancellations}")
print(f"佔三年的百分比：{cancel_percentage:.2f}%")

取消最多的航班號是：76
取消次數：153
佔三年的百分比：0.06%


In [41]:
# 確保FLIGHT_DATE列是日期格式
df['FLIGHT_DATE'] = pd.to_datetime(df['FLIGHT_DATE'])

In [43]:
# 過濾出出發地為OGG的航班
ogg_flights = df[df['ORIGIN'] == 'OGG']

In [45]:
# 確保FLIGHT_DATE是datetime格式
ogg_flights['year'] = ogg_flights['FLIGHT_DATE'].dt.year

# 計算每年每個取消原因的出現次數
cancellation_by_year = ogg_flights.groupby(['year', 'CANCELLATION_CODE']).size().unstack(fill_value=0)

# 計算每年每個取消原因的占比
cancellation_percentage = cancellation_by_year.div(cancellation_by_year.sum(axis=1), axis=0) * 100

C:\Users\user\AppData\Local\Temp\ipykernel_22688\3784888670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ogg_flights['year'] = ogg_flights['FLIGHT_DATE'].dt.year


In [47]:
print(cancellation_percentage)

CANCELLATION_CODE          A         B         C          D
year                                                       
2020               10.994764  1.047120  0.000000  87.958115
2021               91.071429  6.696429  0.446429   1.785714
2022               97.340426  2.127660  0.531915   0.000000


In [53]:
# 過濾出出發地為IAD的航班
iad_flights = df[df['ORIGIN'] == 'IAD']

# 計算每個航空公司的航班次數
carrier_flight_counts = iad_flights.groupby(['OP_CARRIER', 'CARRIER_NAME']).size().reset_index(name='Flight_Count')

# 找出起飛次數最多的航空公司
top_carrier = carrier_flight_counts.sort_values(by='Flight_Count', ascending=False).iloc[0]

In [55]:
# 過濾出被取消的航班
cancelled_iad_flights = iad_flights[iad_flights['CANCELLATION_CODE'].notnull()]

# 計算每個取消原因的出現次數
cancellation_reasons = cancelled_iad_flights['CANCELLATION_CODE'].value_counts()

# 找出最常見的取消原因
top_cancellation_reason = cancellation_reasons.idxmax()

In [57]:
print(f"從IAD起飛架次最多的航空公司是: {top_carrier['CARRIER_NAME']} ({top_carrier['OP_CARRIER']})")
print(f"從IAD起飛航班遭取消的最常見原因是: {top_cancellation_reason}")

從IAD起飛架次最多的航空公司是: AnyCompany Apricot Airlines (AA)
從IAD起飛航班遭取消的最常見原因是: B


In [59]:
# 過濾取消原因為C或D的航班
cancelled_without_CD = cancelled_flights[~cancelled_flights['CANCELLATION_CODE'].isin(['C', 'D'])]

In [61]:
# 找出取消原因為C或D的航班
cancelled_with_CD = cancelled_flights[cancelled_flights['CANCELLATION_CODE'].isin(['C', 'D'])]

# 找出所有航空公司代號
all_carriers = df['OP_CARRIER'].unique()

# 找出有取消C或D原因的航空公司
carriers_with_CD = cancelled_with_CD['OP_CARRIER'].unique()

# 找出從未因C或D取消的航空公司
carriers_without_CD = [carrier for carrier in all_carriers if carrier not in carriers_with_CD]

# 過濾出這些航空公司的數據
flights_without_CD = cancelled_without_CD[cancelled_without_CD['OP_CARRIER'].isin(carriers_without_CD)]

In [63]:
# 計算取消的航班數量
cancelled_counts = flights_without_CD.groupby('OP_CARRIER').size().reset_index(name='Cancelled_Flight_Count')

print(cancelled_counts)

  OP_CARRIER  Cancelled_Flight_Count
0         RA                    5428
